**About** : This notebook is used to infer models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

In [ ]:
import os
import re
import cv2
import sys
import glob
import json
import time
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
from utils.logger import Config, upload_to_kaggle

from params import *
from data.preparation import *

from model_zoo.models import define_model
from utils.metrics import *
from utils.torch import load_model_weights
from utils.plots import plot_sample
from inference.main import uniform_soup

## Expes

In [ ]:
EXP_FOLDER = "../logs/2023-04-17/42/"  # 0.7265 / x5 0.7273 / MTx10 0.7274

EXP_FOLDER = "../logs/2023-04-21/31/"
# EXP_FOLDER = "../logs/2023-04-19/6/"

EXP_FOLDER = "../logs/2023-04-23/27/"   # 0.7302 / DIST 0.7341 / DISTx10 0.7310
EXP_FOLDER = "../logs/2023-04-23/28/"   # 0.7295 / DIST 0.7341 / DISTx10 0.7348

EXP_FOLDER = "../logs/2023-04-24/9/"

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))

In [ ]:
df = prepare_data(DATA_PATH, config.processed_folder)

In [ ]:
if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])

## Inference

### Preprocessing

In [ ]:
from tflite.prepro import *

In [ ]:
landmarks = np.concatenate(KEPT_LANDMARKS)
type_embed = np.zeros(1000)
start = 0
for subset, idx in zip(KEPT_LANDMARKS, MAPPING):
    print(subset, idx)
    type_embed[start: start + len(subset)] = idx
    start += len(subset)

type_embed = type_embed[type_embed > 0]

type_embed = np.concatenate([type_embed, np.array([idx] * len(TO_AVG))])

print("\nn_landmarks :", len(type_embed))

In [ ]:
MAX_LENS = {
    "torch_12/": 25,
    "torch_16/": 30,
}

In [ ]:
prepro = Preprocessing(type_embed, max_len=MAX_LENS[config.processed_folder], model_max_len=config.max_len)
config.max_len

In [ ]:
inp = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")
prepro_tf = PreprocessingTF(type_embed, max_len=MAX_LENS[config.processed_folder], model_max_len=config.max_len)

In [ ]:
preds = []
times = []

# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(100)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)
    
    x = torch.from_numpy(data)
    x_ = tf.constant(data)

    x_torch = prepro(x)
    x_tf = prepro_tf(x_)
    
    print(np.abs(x_tf.numpy() - x_torch.numpy()).max())

    break

### Model

In [ ]:
from tflite.models import Model
from utils.torch import count_parameters

In [ ]:
DIST = config.mt_config['distill']
DIST = True

model = Model(
    type_embed,
    embed_dim=config.embed_dim,
    transfo_dim=config.transfo_dim if not DIST else 512,
    dense_dim=config.dense_dim if not DIST else 192,
    transfo_layers=config.transfo_layers  if not DIST else 2,
    transfo_heads=config.transfo_heads,
    drop_rate=config.drop_rate,
    num_classes=config.num_classes,
    max_len=config.max_len,
).cpu().eval()

config.embed_dim, config.transfo_dim

In [ ]:
N_SOUP = 0
TEACHER = False

In [ ]:
if DIST:
    pred_oof = np.load(EXP_FOLDER + "pred_oof_dist.npy")
elif TEACHER:
    pred_oof = np.load(EXP_FOLDER + "pred_oof_teach.npy")
else:
    try:
        pred_oof = np.load(EXP_FOLDER + "pred_oof_inf.npy")
    except:
        pred_oof = np.load(EXP_FOLDER + "pred_oof.npy")

score = accuracy(df['target'], pred_oof)
print(f"-> CV acc : {score:.4f}")

In [ ]:
if N_SOUP:
    if TEACHER:
        weights = [EXP_FOLDER + f"{config.name}_teacher_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
    elif DIST:
        weights = [EXP_FOLDER + f"{config.name}_distilled_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
    else:
        weights = [EXP_FOLDER + f"{config.name}_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
    print("-> Soup :", [w.split('/')[-1] for w in weights])
    model = uniform_soup(model, weights)

else:
    try:
    #     model = load_model_weights(model, EXP_FOLDER + f"{config.name}_fullfit_0.pt")
    #     model = load_model_weights(model, EXP_FOLDER + f"{config.name}_teacher_fullfit_0.pt")
#         model = load_model_weights(model, EXP_FOLDER + f"{config.name}_0.pt")
        model = load_model_weights(model, EXP_FOLDER + f"{config.name}_distilled_0.pt")
    
#         weights = [EXP_FOLDER + f"{config.name}_0_{ep}.pt" for ep in range(config.epochs - 10, config.epochs + 1)]
#         print("-> Soup :", [w.split('/')[-1] for w in weights])
#         model = uniform_soup(model, weights)
    
    except: # FileNotFoundError:
        print('Not loading weights !')

In [ ]:
df = df[df['fold'] == 0].reset_index(drop=True)
pred_val = np.load(EXP_FOLDER + "pred_val_0.npy")
print('Ref val acc :', accuracy(df['target'], pred_val))

In [ ]:
preds = []
times = []

# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(100)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)
    
    x = torch.from_numpy(data)

    t0 = time.time()
    x = prepro(x)
    y = model(x)
    preds.append(y.detach().cpu().numpy().flatten())
    t1 = time.time()
    
    times.append((t1 - t0) * 1000)
#     break

In [ ]:
print(f'Runtime : {np.mean(times) :.1f}ms')

In [ ]:
preds = np.stack(preds)

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

#### Nobuco

In [ ]:
import nobuco
import tensorflow_addons as tfa
from nobuco import ChannelOrder, ChannelOrderingStrategy

In [ ]:
@nobuco.converter(torch.nn.functional.mish, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def mish(input: torch.Tensor, inplace: bool = False):
    return lambda input, inplace=False: tfa.activations.mish(input)

In [ ]:
@nobuco.converter(torch.Tensor.long, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def long(input: torch.Tensor, inplace: bool = False):
    return lambda input, inplace=False: tf.cast(input, tf.int64)

In [ ]:
@nobuco.converter(torch.Tensor.int, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def int(input: torch.Tensor, inplace: bool = False):
    return lambda input, inplace=False: tf.cast(input, tf.int32)

In [ ]:
@nobuco.converter(torch.Tensor.amax, channel_ordering_strategy=ChannelOrderingStrategy.FORCE_PYTORCH_ORDER)
def amax(input: torch.Tensor, dim=None, keepdim=False):
    return lambda input, axis: tf.reduce_max(input, axis=dim)

In [ ]:
@nobuco.converter(torch.gather, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def gather(input, dim, index):
    return lambda input, dim, index: tf.gather(input, index)

In [ ]:
@nobuco.converter(torch.zeros, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def zeros(*size):
    print(size)
    return lambda size: tf.zeros(size)

In [ ]:
path = df['path'][0]
pq, data = load_relevant_data_subset(path)
inp = torch.from_numpy(prepro_tf(data).numpy()).contiguous()
inp.size()

In [ ]:
keras_model = nobuco.pytorch_to_keras(
    model,
    args=[inp],
    input_shapes={inp: (None, 5, 100)},
    inputs_channel_order=ChannelOrder.PYTORCH,
    outputs_channel_order=ChannelOrder.TENSORFLOW
)

In [ ]:
# keras_model.summary()

In [ ]:
preds = []
for i in tqdm(range(100)):
    path = df['path'][i]
    pq, data = load_relevant_data_subset(path)
    x = prepro_tf(data)
    y = keras_model(x)
    preds.append(y.numpy()[0])
#     break

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

### Prepro + model

In [ ]:
class TFLiteModel(tf.keras.Model):
    def __init__(self, prepro, model):
        super(TFLiteModel, self).__init__()
        self.prepro = prepro
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')])
    def call(self, inputs=None):
        x = self.prepro(tf.cast(inputs, dtype=tf.float32))
        y = self.model(x)

        return {'outputs': y}

In [ ]:
prepro_tf = PreprocessingTF(type_embed, max_len=MAX_LENS[config.processed_folder], model_max_len=config.max_len)

In [ ]:
tflite_keras_model = TFLiteModel(prepro_tf, keras_model)

In [ ]:
preds = []

# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(1000)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)

    y = tflite_keras_model(data)
    
    preds.append(y['outputs'].numpy()[0])
#     break

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

In [ ]:
tflite_keras_model.save(EXP_FOLDER + 'model_keras')

### TfLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(EXP_FOLDER + "model_keras")

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

# converter.target_spec.supported_ops = [
#     tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
#     tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
# ]

tflite_model = converter.convert()

with open(EXP_FOLDER + 'model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
from tflite_runtime.interpreter import Interpreter

interpreter = Interpreter(EXP_FOLDER + "model.tflite")

prediction_fn = interpreter.get_signature_runner("serving_default")

output = prediction_fn(inputs=data)
output['outputs'].max()

In [ ]:
preds = []
times = []
# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(1000)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)

    t0 = time.time()
    output = prediction_fn(inputs=data)
    t1 = time.time()

    preds.append(output['outputs'])
    times.append((t1 - t0) * 1000)
    
#     break

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

In [ ]:
accuracy(df['target'].head(len(preds)), pred_val[:len(preds)])

In [ ]:
print(f'-> Runtime : {np.mean(times) :.1f}ms')

if np.mean(times) > 100:
    print("\n WARNING ! Runtime must be < 100 ms !")

### Size & upload

In [ ]:
size = os.path.getsize(EXP_FOLDER + 'model.tflite') / np.power(1024, 2)
print(f"-> Model size : {size:.3f} Mo")

assert size < 40, "Model size must be < 40 Mo !"

In [ ]:
upload_to_kaggle([EXP_FOLDER], "/workspace/datasets/islr_weights_1/", "ISLR Models", update_folders=False)

Done ! 